In [27]:
import my_toolbox as tb
import numpy as np
import calibration as calib
import main
import importlib

importlib.reload(main)

main_path = "C:/Users/Ben/My Drive/PhD/PhD Year 3/3rd Year Paper/Model/My Code/MH_Model/my_code/model_uncert/"
age_type_input_path = main_path + "input/50p_age_moms/"
trans_path = age_type_input_path + "MH_trans_by_MH_clust_age.csv"  
# my_grid = np.array([5.0, 10.0, 15.0, 20.0])
my_grid = np.linspace(0.5, 20, 100)
my_grid = np.log(my_grid)
myPars = main.pars_factory(main_path, my_lab_fe_grid = my_grid, H_trans_path = trans_path) 




Using health transition matrix from: C:/Users/Ben/My Drive/PhD/PhD Year 3/3rd Year Paper/Model/My Code/MH_Model/my_code/model_uncert/input/50p_age_moms/MH_trans_by_MH_clust_age.csv


In [28]:
importlib.reload(calib)
mu = 2.0
sigma = .3

mu_mom_targ = 2.2
sigma_mom_targ = .299

# define lambda function to minimize
mu_mom_err = lambda new_mu: calib.w0_mu_mom_giv_mu(myPars, new_mu) - mu_mom_targ
sig_mom_err = lambda new_sig: calib.w0_sigma_mom_giv_sigma(myPars, new_sig) - sigma_mom_targ

calib_mu = -999  # initialize
mu_tol = 0.001
mu_max = 5.0
mu_min = 0.0

calib_sigma = -999  # initialize
sig_tol = 0.001

max_iter = 20
mu_iters = 0
while abs(mu_mom_err(myPars.lab_fe_tauch_mu)) > mu_tol and mu_iters < max_iter:
    mu_mid_pt = (mu_max + mu_min) / 2
    print("***Calibrating Sigma***")

    sig_iters = 0
    sig_max = 1.0
    sig_min = 0.001
    while abs(sig_mom_err(myPars.lab_fe_tauch_sigma)) > sig_tol and sig_iters < max_iter:
        sig_mid_pt = (sig_max + sig_min) / 2
        sig_mom_err_mid = sig_mom_err(sig_mid_pt)
        if sig_mom_err_mid > 0:
            sig_max = sig_mid_pt
        else:
            sig_min = sig_mid_pt
        sig_iters += 1

        calib_sigma = sig_mid_pt
        my_lab_weights = myPars.lab_fe_weights

    print(f"sigma: {calib_sigma}, sigma mom diff: {sig_mom_err(calib_sigma)}")
    print(f"Weights: {my_lab_weights}")
    print()
    mu_mom_err_mid = mu_mom_err(mu_mid_pt)
    if mu_mom_err_mid > 0:
        mu_max = mu_mid_pt
    else:
        mu_min = mu_mid_pt
    mu_iters += 1
    calib_mu = mu_mid_pt
    my_lab_weights = myPars.lab_fe_weights
    print(f"mu: {calib_mu}, mu mom diff: {mu_mom_err(calib_mu)}")
    print(f"Weights: {my_lab_weights}")
    print()

print(f"After {mu_iters} iterations")
print(f"Calibrated mu: {calib_mu}, myPars mu: {myPars.lab_fe_tauch_mu}, model mu moment: {calib.w0_mu_moment(myPars)}, target mu moment: {mu_mom_targ}")
print(f"Calibrated sigma: {calib_sigma}, myPars sigma: {myPars.lab_fe_tauch_sigma}, model sigma moment: {calib.w0_sigma_moment(myPars)}, target sigma moment: {sigma_mom_targ}")
print()


***Calibrating Sigma***
sigma: 0.29367578124999993, sigma mom diff: 0.000994150483237699
Weights: [1.08149021e-02 1.11022143e-01 2.53279260e-01 2.65356739e-01
 1.82023740e-01 9.81632661e-02 4.59888814e-02 1.98261191e-02
 8.14320261e-03 3.25667532e-03 1.28605106e-03 5.06106822e-04
 1.99703648e-04 7.93361281e-05 3.18191519e-05 1.29070434e-05
 5.30143758e-06 2.20648576e-06 9.30941352e-07 3.98227983e-07
 1.72716517e-07 7.59422040e-08 3.38453654e-08 1.52854044e-08
 6.99350367e-09 3.24056226e-09 1.52024870e-09 7.21830040e-10
 3.46767060e-10 1.68492109e-10 8.27787838e-11 4.11072287e-11
 2.06270556e-11 1.04556364e-11 5.35216316e-12 2.76589862e-12
 1.44273482e-12 7.59281527e-13 4.03121980e-13 2.15827356e-13
 1.16462395e-13 6.35047570e-14 3.47499807e-14 1.92068583e-14
 1.06581410e-14 5.99520433e-15 3.44169138e-15 1.88737914e-15
 1.11022302e-15 6.66133815e-16 3.33066907e-16 2.22044605e-16
 1.11022302e-16 1.11022302e-16 0.00000000e+00 1.11022302e-16
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.

In [29]:
calib_mu = myPars.lab_fe_tauch_mu
calib_sigma = myPars.lab_fe_tauch_sigma
calib_sigma = 1.0

tauch_weights, tauch_grid = tb.Taucheniid(calib_sigma, len(my_grid), Nsd = 3, mean = calib_mu, state_grid = my_grid)
print(f"Tauch weights: {tauch_weights}")
print(f"Tauchen grid: {tauch_grid}")

def tauch_mean(tauch_weights, tauch_grid):
    return np.sum(tauch_weights*tauch_grid)

def tauch_sd(tauch_weights, tauch_grid):
    return np.sqrt(np.sum(tauch_weights*(tauch_grid - tauch_mean(tauch_weights, tauch_grid))**2))

print(f"Calib_mu: {calib_mu}")
print(f"Tauch mean: {tauch_mean(tauch_weights, tauch_grid)}")
print(f"Calib_sigma: {calib_sigma}")
print(f"Tauch sd: {tauch_sd(tauch_weights, tauch_grid)}")

Tauch weights: [0.00298239 0.00482971 0.00724133 0.00939138 0.01120472 0.01268192
 0.01385329 0.01475843 0.01543722 0.01592624 0.01625756 0.01645865
 0.01655268 0.01655913 0.01649421 0.01637146 0.0162021  0.0159955
 0.0157594  0.01550026 0.01522345 0.0149334  0.01463381 0.01432774
 0.01401773 0.01370587 0.01339389 0.0130832  0.01277496 0.01247012
 0.01216943 0.0118735  0.01158281 0.01129773 0.01101854 0.01074544
 0.01047859 0.01021805 0.00996388 0.00971608 0.00947462 0.00923947
 0.00901054 0.00878776 0.00857101 0.0083602  0.00815519 0.00795587
 0.0077621  0.00757375 0.00739068 0.00721276 0.00703985 0.0068718
 0.00670849 0.00654979 0.00639554 0.00624564 0.00609994 0.00595833
 0.00582067 0.00568686 0.00555676 0.00543028 0.00530729 0.0051877
 0.00507139 0.00495827 0.00484823 0.00474118 0.00463704 0.0045357
 0.00443708 0.0043411  0.00424767 0.00415673 0.00406818 0.00398197
 0.00389801 0.00381624 0.00373659 0.003659   0.0035834  0.00350973
 0.00343795 0.00336798 0.00329978 0.00323329 0.0031